In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# Example MNIST CNN Classifier

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)        # 1 input channel, 6 output channels, 3x3 kernel
        self.conv2 = nn.Conv2d(6, 16, 3)       # 6 input channels, 16 output channels, 3x3 kernel
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 16 channels * 6x6 remaining after pooling
        self.fc2 = nn.Linear(120, 84)          # Basic input to output NN layer
        self.fc3 = nn.Linear(84, 10)

        
    def forward(self, x):
        # Pool Convolutional Layers
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        # Flatten to 1-dimension
        x = x.view(-1, self.num_flat_features(x))
        
        # Push through standard NN layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    
    def num_flat_features(self, x):
        size = x.size()[1:]  # First dimension is batch size
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [4]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
ex_input = torch.randn(1, 1, 32, 32) # Batch, Inputs, Height, Width

In [6]:
# Parameters of CNN automatically detected by the module class
# Parameters are a subset of the Tensor class

params = list(net.parameters())
for layer in params:
    print(layer.size())

torch.Size([6, 1, 3, 3])
torch.Size([6])
torch.Size([16, 6, 3, 3])
torch.Size([16])
torch.Size([120, 576])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [14]:
# Get output of the CNN

output = net(ex_input)
output

tensor([[-0.1087, -0.0846,  0.0069,  0.0573, -0.0207,  0.1358, -0.0682, -0.1119,
          0.0641,  0.0469]], grad_fn=<AddmmBackward>)

In [15]:
# Example y batch
# Batch Size * Number of Classes

target = torch.tensor([0.,0.,0.,1.,0.,0.,0.,0.,0.,0.]).view(1, -1)
target.shape

torch.Size([1, 10])

In [16]:
# Calculate loss

criterion = nn.MSELoss()
loss = criterion(output, target)
loss

tensor(0.0950, grad_fn=<MseLossBackward>)

In [17]:
# Set gradients to zero, otherwise the gradient values will accumulate
net.zero_grad()
print(net.conv1.bias.grad)

# Get the gradients based on the loss value
loss.backward()
print(net.conv1.bias.grad)

tensor([0., 0., 0., 0., 0., 0.])
tensor([ 0.0010, -0.0064, -0.0020,  0.0026,  0.0023,  0.0043])


In [21]:
# Manually update whatever tensors you want
# Tensor.data.sub_(Tensor.grad.data)
# Or in whatever method needed

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [22]:
import torch.optim as optim

# Create optimizer and pass it the tensors
optimizer = optim.SGD(net.parameters(), lr=0.01)

# Calculate loss
optimizer.zero_grad()
output = net(ex_input)
loss = criterion(output, target)
loss.backward()

# Updates the tensors with the logic of the optimizer
optimizer.step()